In [ ]:
### 1. Library Imports and Excel File Read-in

import pandas as pd
from datetime import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

df = pd.read_excel('/Users/mburley/Desktop/2022 Oakley Data/2022_Oakley_Transactions_Master.xlsx') #Reads in data from excel file
print(df.tail(5))

In [ ]:
### 2. Drop Columns 

df = df.drop(['Date Bought 2021', 'Week', 'Transfer Fees (Paypal)', 'Parts','Unnamed: 13', 'Unnamed: 15', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Extra Charges', 'SALES', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 30', 'Unnamed: 31', 'Personal Pairs', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 37', 'Additional Write-offs'], 1)
df

In [ ]:
### 3. List Column Data Types

# Convert "Paid' column to float64 by first changing NaN to 0
df['Paid'].fillna(0).astype(float)

# Convert 'Sale Price' column to float64 by first changing NaN to 0
#print(df.loc[pd.to_numeric(df['Sale Price'], errors='coerce').isnull()])
#pd.to_numeric(df['Sale Price']).astype(int)
df["Sale Price"] = df["Sale Price"].astype(str).str.strip().replace("",0).astype(float)

# Convert "Date Bought" column to Datetime
df['Date Bought 2022'].apply(pd.to_datetime)

# Convert "Date Sold" column to Datetime
df['Date Sold'].apply(pd.to_datetime)

# List Data Types
df.dtypes

In [ ]:
### 4. Describe Data

df.describe()

In [ ]:
### 5. Find Sum Totals for "Paid", "Sale Price", "Profit", "Shipping Fees" Columns

print("Total Spent on Inventory Acqiusition: $" + df['Paid'].sum().astype(str))
print("Total Amount Sold: $" + df['Sale Price'].sum().astype(str))
print("Total Net Profit: $" + df['Profit*'].sum().astype(str))
print("Total Spent on Shipping Fees: $" + df['Shipping Fees*'].sum().astype(str))

In [ ]:
### 6. Top 10 Most Popular Frame Styles

# Count 'Frame Style' values based on 'Sale Price' entries counted up, and sort the counted 'Paid' values highest to lowest, display Top 10
freq_frames_sold = df.groupby('Frame Style').count().sort_values(['Sale Price'], ascending=False)['Sale Price'].head(10)
display(freq_frames_sold)

### Graph:

# Create Seaborn Barplot and rename y label
plt.figure(figsize=(15,6))
# x = 'Values Displayed for x axis' and y = 'Values Displayed for y axis' and data = location data is being pulled in from
chart6 = sns.barplot(x = 'Frame Style', y = 'Sale Price', data = freq_frames_sold.reset_index(), color = "green")
plt.xlabel('Frame style', size = 12)
plt.ylabel('Number Sold', size = 12)
chart6.set_title('Top 10 Most Popular Frame Styles', size = 12)
# Rotates x axis values
chart6.set_xticklabels(chart6.get_xticklabels(), rotation=20)
plt.show()

In [ ]:
### 7. Top 10 Most Expensive Items Sold

# .nlargest(how many largest values shown, in which columns)
df7 = df.nlargest(10, ['Sale Price'], keep = 'all')
display(df7)

### Graph:

plt.figure(figsize=(15,6))
chart7 = sns.barplot(x = 'Frame Style', y = 'Sale Price', data = df7, color = "Blue")
plt.xlabel('Frame Style', size = 12)
plt.ylabel('Sale Price', size = 12)
chart7.set_title('Top 10 Most Expensive Items Sold', size = 12)
chart7.set_xticklabels(chart7.get_xticklabels(), rotation=20)
plt.show()

In [ ]:
### 8. Most Common Purchasing Website

# Display 'Website' and count(), and display values in the 'Paid' column from high to low, ['Paid'] at end means display ONLY that col
freq_websites = df.groupby('Website').count().sort_values(['Paid'], ascending = False)['Paid']
display(freq_websites)
# df['Location'].count()

### Graph:

# Create Seaborn Barplot and rename y label
plt.figure(figsize=(15,6))
# x = 'Values Displayed for x axis' and y = 'Values Displayed for y axis' and data = location data is being pulled in from
chart8 = sns.barplot(x = 'Website', y = 'Paid', data = freq_websites.reset_index(), color = "red")
plt.xlabel('Website', size = 12)
plt.ylabel('Number Purchased', size = 12)
chart8.set_title('Most Common Purchasing Website', size = 12)
# Rotates x axis values
chart8.set_xticklabels(chart8.get_xticklabels(), rotation = 20)
plt.show()

In [ ]:
### 9. Compute Total Amount Purchased and Sold from Each Website

# Display Amount Purchased from each Website
sum_per_website = df.groupby(['Website']).sum().sort_values(['Paid'], ascending = False)['Paid']
display(sum_per_website)

# Display Amount Sold from each Website
sum_sold_website = df.groupby(['Website']).sum().sort_values(['Sale Price'], ascending = False)['Sale Price']
display(sum_sold_website)

### Graph: 

# Create df with website and summed values
df9 = df.groupby(['Website']).sum().sort_values(['Paid'], ascending = False).drop(['Total', 'Profit*', 'Shipping Fees*'], 1)
display(df9)

In [ ]:
### 10. Count the Number of Items Bought and Sold

total_bought = df['Paid'].count().astype(str)
print("Total Number of Items bought: " + total_bought)

total_sold = df['Sale Price'].count().astype(str)
print("Total Number of Items Sold: " + total_sold)

print("\n")

### 10.5. Count the Number of Complete Pairs Bought and Sold

total_complete_bought = df.dropna(axis=0, subset=['Frame Color', 'Lenses']).count().astype(str)['Paid']
print("Total Number of Complete Pairs bought: " + total_complete_bought)

total_complete_sold = df.dropna(axis=0, subset=['Frame Color', 'Lenses']).count().astype(str)['Sale Price']
print("Total Number of Complete Pairs sold: " + total_complete_sold)


In [ ]:
### 11. Most profitable Frame Style Based on Avg Sale of all counted "Frame Styles" (Tests profitability vs. availibility)
## Ex. (M Frame Avg Profit) * (Number of M Frames Sold / Total Frames Sold)

# Create var = Grouped Frame Style Values
group = df.groupby('Frame Style', sort = False)
#group

# Using group var, find mean of profit values for each frame, times counted frame sold / total sold like Example above and sort
weighted_profit = group['Profit*'].sum()*(group['Profit*'].count().astype(float)/total_sold.astype(float))
weighted_profit.sort_values(ascending = False).head(15)


In [ ]:
### 12. Most Popular Day of Week for Buying 2022

## Most Popular Buying Day
# Insert Weekday Column for day of the week bought in form df.insert(position, 'name_of_column', what you're doing)
#df.insert(1, 'Weekday Bought', df['Date Bought'].dt.day_name())
# Create Weekday Bought Column  by converting 'Date Bought' to day of week name
df['Weekday Bought'] = df['Date Bought 2022'].dt.day_name()
#df.tail(10)

# Count each day of week value in "Weekday Bought" and list high to low
df12 = df.groupby('Weekday Bought').count().sort_values(['Paid'], ascending = False)['Paid']
display(df12)

### Graph:
# Reset Index and do Custom Sorting
df12 = df12.reset_index()
df12['Weekday Bought'] = pd.Categorical(df12['Weekday Bought'],
                            categories=['Monday', 'Tuesday', 'Wednesday',
                                        'Thursday', 'Friday', 'Saturday', 'Sunday'],
                            ordered=True)
df12 = df12.sort_values('Weekday Bought')

# Generate Line Graph with x = Weekday Bought (Mon - Sun) and y = Counted Paid Values
plt.figure(figsize=(15,6))
chart12 = sns.lineplot(x = 'Weekday Bought', y = 'Paid', data = df12, color = "green")
plt.xlabel('Weekday Bought', size = 12)
plt.ylabel('Number Bought', size = 12)
chart12.set_title('Most Popular Day of Week for Buying', size = 12)
plt.show()

In [ ]:
### 13. Compute Profit Acquired on Each Day of Week from above Data -> Display 'Weekday Bought' and "Profit*"

# as_index = FALSE says we don't want to set the column ID as the index!
df.groupby(['Weekday Bought'], as_index = False)['Profit*'].sum().sort_values(['Profit*'], ascending = False)


In [ ]:
### 14. Most Popular Day of Week for Selling

## Most Popular Selling Day
# Insert Weekday Column for day of the week Sold in form df.insert(position, 'name_of_column', what you're doing)
df['Weekday Sold'] = df['Date Sold'].dt.day_name()
#df.tail(10)

# Count each day of week value in "Weekday Bought" and list high to low
df14 = df.groupby('Weekday Sold').count().sort_values(['Sale Price'], ascending = False)['Sale Price']
display(df14)

### Graph:
# Reset Index and do Custom Sorting
df14 = df14.reset_index()
df14['Weekday Sold'] = pd.Categorical(df14['Weekday Sold'],
                            categories=['Monday', 'Tuesday', 'Wednesday',
                                        'Thursday', 'Friday', 'Saturday', 'Sunday'],
                            ordered=True)
df14 = df14.sort_values('Weekday Sold')

# Generate Line Graph with x = Weekday Sold (Mon - Sun) and y = Counted Sale Price Values
plt.figure(figsize=(15,6))
chart14 = sns.lineplot(x = 'Weekday Sold', y = 'Sale Price', data = df14, color = "blue")
plt.xlabel('Weekday Sold', size = 12)
plt.ylabel('Number Sold', size = 12)
chart14.set_title('Most Popular Day of Week for Selling', size = 12)
plt.show()


In [ ]:
### 15. Compute Amount Sold on Each Day of Week from above Data -> Display 'Weekday Sold' and 'Sale Price'

# as_index = FALSE says we don't want to set the column ID as the index!
df15 = df.groupby(['Weekday Sold'], as_index = False)['Sale Price'].sum().sort_values(['Sale Price'], ascending = False)
display(df15)

### Graph:
# Reset Index and do Custom Sorting
df15 = df15.reset_index()
df15['Weekday Sold'] = pd.Categorical(df15['Weekday Sold'],
                            categories=['Monday', 'Tuesday', 'Wednesday',
                                        'Thursday', 'Friday', 'Saturday', 'Sunday'],
                            ordered=True)
df15 = df15.sort_values('Weekday Sold')

# Generate Line Graph with x = Weekday Sold and y = Summed Sale Price Values
plt.figure(figsize=(15,6))
chart15 = sns.lineplot(x = 'Weekday Sold', y = 'Sale Price', data = df15, color = "red")
plt.xlabel('Weekday Sold', size = 12)
plt.ylabel('Number Sold', size = 12)
chart12.set_title('Amount Sold on Each Day of Week', size = 12)
plt.show()

In [ ]:
### 16. Best Week For Buying

# Best Week for Buying (Week with Most Bought)
# Creates "Week Bought" column and sets it equal to the week number from the date of the item sold
df['Week Bought'] = df['Date Bought 2022'].dt.strftime('%U')
df.tail(30)
df.groupby('Week Bought').sum().sort_values(['Paid'], ascending = False).head(10)['Paid']


In [ ]:
### 17. Best Month For Buying 

# Best Month for Buying
# Creates "Month Bought" column and sets it equal to the month number from the date of the item bought then converts to Month Names in datetime dtype
df['Month Bought'] = pd.to_datetime(df['Date Bought 2022'], format='%m').dt.month_name().str.slice(stop=12)

# Group "Month Bought" then sum, sort and display "Paid" values
df.groupby('Month Bought').sum().sort_values(['Paid'], ascending = False)['Paid']


In [ ]:
### 18. Best Month For Selling 

# Best Month for Selling
# Creates "Month Sold" column and sets it equal to the month number from the date of the item sold then converts it to a Month Name and datetime dtype
df['Month Sold'] = pd.to_datetime(df['Date Sold'], format='%m').dt.month_name()

# Group "Month Sold" then sum, sort and display "Sale Price" values
df.groupby('Month Sold').sum().sort_values(['Sale Price'], ascending = False).head(10)['Sale Price']


In [ ]:
### 19. Compute the Avg Number of Items Bought/Sold Per Day -> Total Bought/Sold(Prompt 10) / Total Days

# Compute Total Number of Days
least_recent_date = dt(2022, 1, 1)
most_recent_date = df['Date Bought 2022'].max()
#print(most_recent_date)
time_delta = most_recent_date - least_recent_date
#print(time_delta)

# Convert time_delta from timedelta to float
total_days = time_delta.total_seconds() / (24 * 60 * 60)
#print(total_days)


## Avg Number of Pairs Bought Per Day
bought_2022 = total_bought.astype(float) - 161
#print(bought_2022)
avg_bought_per_day = (bought_2022.astype(float) / total_days).astype(str)
print("The Average Number of Items Bought per day is: " + avg_bought_per_day)


## Avg Number of Pairs Sold Per Day
#print(total_sold)
avg_sold_per_day = (total_sold.astype(float) / total_days).astype(str)
print("The Average Number of Items Sold per day is: " + avg_sold_per_day)


In [ ]:
### 20. Count Total Number of Pairs Bought from each State

# Display Amount Purchased from each Website
pairs_from_state = df.value_counts(['State Bought'])
display(pairs_from_state.head(10))


In [ ]:
### 21. Coumpute Total Amount Bought from each State

# Group "State Bought" then sum, sort and display "Paid" values
df.groupby('State Bought').sum().sort_values(['Paid'], ascending = False).head(10)['Paid']


In [ ]:
### 22. Compute Total Profit Aquired from each State

# Group "State Bought" then sum, sort and display "Profit*" values
df.groupby('State Bought').sum().sort_values(['Profit*'], ascending = False).head(10)['Profit*']


In [ ]:
### 23. Compute Avg Amount Bought Per Week -> (Total Bought / Num Weeks)

# Compute the total amount bought 
total_bought = df['Paid'].sum()
#print(total_bought)

# Count the number of weeks
df['Week Bought'] = df['Week Bought'].astype(str).str.strip().replace("",0).astype(float)
count_weeks = df['Week Bought'].max()
#print(count_weeks)

# Computation
avg_bought_per_week = (total_bought - 7283.6) / (count_weeks + 1)
print("The average amount bought per week is: $" + avg_bought_per_week.round(2).astype(str))

In [ ]:
### 24. Compute Avg Number of Items bought per week

# Load in total number of pairs bought
total_bought = df['Paid'].count()
#print(total_bought)

# Count total number of weeks
#print(count_weeks)

# Divide total bought divided by total weeks to compute AVG
avg_num_bought_per_week = (total_bought / count_weeks)
#str(avg_num_bought_per_week)
print("The avg number of items purchased per week is: " + avg_num_bought_per_week.round(2).astype(str))

In [ ]:
### 25. Compute Net Profit Margin (Profiability) as a Percentage of Revenue

total_profit = df['Profit*'].sum()
total_profit

total_revenue = df['Sale Price'].sum()
total_revenue

total_ship_costs = df['Shipping Fees*'].sum()
total_ship_costs

# Compute Post Shipping fee Net Profit Margin
profitability = (total_profit - total_ship_costs) / total_revenue
percentage = "{:.2%}".format(profitability)
print('The Net Profit Margin is: ' + percentage)

In [ ]:
### 26. Compute Inventory Assets vs. COGS (Total amount of inventory and total amount sold)

# Create Inventory data frame by selecting rows where the Sale Price column values are Null 
df_inv = df[df['Sale Price'].isnull()]
df_inv.head(10)

# Sum Paid column values in df_inv to get total current inventory assets
total_inv = df_inv['Paid'].sum()
print("The total current inventory asset amount is: $" + total_inv.round(2).astype(str))

# Create COGS data frame by selecting rows where the Sale Price column values are NOT Null 
df_cogs = df[df['Sale Price'].notnull()]
df_cogs.head(10)

# Sum Paid column values in df_inv to get total current inventory assets
total_cogs = df_cogs['Paid'].sum()
print("The total COGS is: $"+ total_cogs.round(2).astype(str))
